# FBA tutorial

Versions: 0.2.0

Author: JV & MRG

This notebook is intended as a tutorial for how to do common genome-scale metabolic modelling simulations.

Installation of required software is assumed, specifically COMETS, COBRAPY and GUROBI

As well as using publicly availabed open source packages, we will also make use of sanchez-lab in house scripts. 
COMETS.py and CAFBAFY.py Please contact authors/githubs to make sure you are using the most up to date version of the modules

This tutorial will be updated over time and will focus on CAFBA, FBA and dFBA simulations

Other FBA-related software that we have used but are beyond the scope of this tutorial:

CARVME: Automated metabolic model construction using whole-genome data.
SYBIL: FBA simulations using R.  GlobalFit: R package for removing erroneous energy generating cycles from genome-scale models. libSBML

For more information go to the cobrapy and comets documentation

## Load Packages And models

In [1]:
import numpy as np #Numpy
import pandas as pd #Pandas
import itertools #itertools
import cobra as c #Cobrapy 
#from comets import *
from CAFBAFY import * 

In [2]:
#So the first thing to do is to load the models. For the tutorial we are going to  work with iJO1366 which is the e.coli pulsihed model. 
E = CAFBA_Model(cobra.io.read_sbml_model('iJO1366.xml'))

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


## Structure of the model class

In [3]:
E.metabolites

[<Metabolite 10fthf_c at 0x7fe321e60130>,
 <Metabolite 12dgr120_c at 0x7fe321e601f0>,
 <Metabolite 12dgr140_c at 0x7fe321e60280>,
 <Metabolite 12dgr141_c at 0x7fe321e603a0>,
 <Metabolite 12dgr160_c at 0x7fe321e60460>,
 <Metabolite 12dgr161_c at 0x7fe321e604c0>,
 <Metabolite 12dgr180_c at 0x7fe321e604f0>,
 <Metabolite 12dgr181_c at 0x7fe321e60520>,
 <Metabolite 12ppd__R_c at 0x7fe321e60550>,
 <Metabolite 12ppd__S_c at 0x7fe321e60580>,
 <Metabolite 13dpg_c at 0x7fe321e605b0>,
 <Metabolite 14dhncoa_c at 0x7fe321e605e0>,
 <Metabolite 14glucan_c at 0x7fe321e60610>,
 <Metabolite 15dap_c at 0x7fe321e60640>,
 <Metabolite 1ddecg3p_c at 0x7fe321e60670>,
 <Metabolite 1hdec9eg3p_c at 0x7fe321e606a0>,
 <Metabolite 1hdecg3p_c at 0x7fe321e606d0>,
 <Metabolite 1odec11eg3p_c at 0x7fe321e60700>,
 <Metabolite 1odecg3p_c at 0x7fe321e60730>,
 <Metabolite 1pyr5c_c at 0x7fe321e60760>,
 <Metabolite 5dh4dglc_c at 0x7fe321e60790>,
 <Metabolite 2tdec7eg3p_c at 0x7fe321e607c0>,
 <Metabolite 5drib_c at 0x7fe321e60

In [4]:
#So lets first go through the structure of the model class.If you run each line in a seperate block you can see the outpput
#Each Model cohttp://localhost:8890/notebooks/Comets_test.ipynbntain a bunch of objects associated with them. 

#0 The model ID
E.id

### The most important objects are
#1 the list of reactions
E.reactions
#2 the list of metabolites
E.metabolites
#3 the list of genes
E.genes


###There are a couple of optional objects
#4 A list of compartments - metabolites are sometimes assigned to a compartment (peripslams, cytoplasm, extracellular etc). 
# Most models contain this information
E.compartments
#5 A list of reaction groups - reactions are sometimes assigned to a groups representing metabolic modules. (transporter, glycolysis etc)
# Many models do not contain this information
E.groups

### There are also some objects associated with the optimizer(the programm used to do linear optimization)
# A solver (by defualt we tend to use gurobi, but you can use glpk or cplex)
E.solver
#7. A tolerance this set the threshold for optimizer precisions. I.e fluxes lower than the tolerance are ignored
#E.tolerancegroups


In [5]:
E.id

'iJO1366'

In [6]:
E.compartments

{'c': 'cytosol', 'e': 'extracellular space', 'p': 'periplasm'}

### Reactions

In [19]:
#There are a few different types of reactions in model.

#Most reactions are simple metabolic reactions. They can be irreversable like
E.reactions.PYK

#Or reversible like 
E.reactions.ACKr

#Exchange reactions are used to import and remove metabolites  They are associated with uptake and secretions and 
#and contain only a single metabolite
#For biggs models an exchange reaction is given by the 'EX' identifier. Typically a negative 
#flux through an exchange reaction indicates uptake and a positive flux indicates secretions
E.reactions.EX_glc__D_e

#There are also sink reactions. They are similar to EX_ reactions but apply to intracellular metabolites. 
# These reactions are added in the gapfilling process to make sure model is not blocked by intracellular metabolites
# whose pathways are incomple. Different models use different naming conventions but typical ones are 'sink' 'sk_' or 'DM_'
E.reactions.DM_4crsol_c

#Finally there are two 'unique' reactions. The ATPM requirement which consumes ATP and typically
# has a fixed lower bound that is empiricially measured. You will often find that
# this introduce a second constraint that leads to weakly non linear relationships between 
# uptake and biomass (When resources are set high it  has minimal effect)
E.reactions.ATPM

#The biomass reactions often given using the identifier 'BIOMASS' or growth.  Some models 'such as e.coli' contain
#Multiple biomass reactions. It is based on the empirically measured composition of cellular biomas
E.reactions.BIOMASS_Ec_iJO1366_core_53p95M

#The biomass reaction is just like any other reaction except it has an 'objective coefficent' of 1
# Whilst by default most over reactions have an objective coefficient of 0. In theory you could 
#define your objective function as any linear combination of fluxes by tuning the objective coefficeints
#for the e.coli model  The “wildtype” biomass reaction contains the precursors to all the typical wild-type cellular components of E. coli,
# while the “core” biomass reaction contains the precursors only to essential component. We typically use the latter
# as it tend to avoid False-negative growth
E.reactions.BIOMASS_Ec_iJO1366_core_53p95M.objective_coefficient # the defautl biomass function
E.reactions.BIOMASS_Ec_iJO1366_WT_53p95M.objective_coefficient # the wt biomass function


0

In [7]:
#Or reversible like 
E.reactions.ACKr

Reaction identifier,ACKr
Name,Acetate kinase
Memory address,0x7fe3246309d0
Stoichiometry,ac_c + atp_c <=> actp_c + adp_c Acetate + ATP C10H12N5O13P3 <=> Acetyl phosphate + ADP C10H12N5O10P2
GPR,b3115 or b2296 or b1849
Lower bound,-1000.0
Upper bound,1000.0


In [8]:
E.reactions.PYK


Reaction identifier,PYK
Name,Pyruvate kinase
Memory address,0x7fe35190f250
Stoichiometry,adp_c + h_c + pep_c --> atp_c + pyr_c ADP C10H12N5O10P2 + H+ + Phosphoenolpyruvate --> ATP C10H12N5O13P3 + Pyruvate
GPR,b1854 or b1676
Lower bound,0.0
Upper bound,1000.0


In [23]:
E.reactions.BIOMASS_Ec_iJO1366_core_53p95M

Reaction identifier,BIOMASS_Ec_iJO1366_core_53p95M
Name,E. coli biomass objective function (iJO1366) - core - with 53.95 GAM estimate
Memory address,0x07f1cd83570f0
Stoichiometry,0.000223 10fthf_c + 2.6e-05 2fe2s_c + 0.000223 2ohph_c + 0.00026 4fe4s_c + 0.513689 ala__L_c + 0.000223 amet_c + 0.295792 arg__L_c + 0.241055 asn__L_c + 0.241055 asp__L_c + 54.124831 atp_c + 0.0001... 0.000223 10-Formyltetrahydrofolate + 2.6e-05 [2Fe-2S] iron-sulfur cluster + 0.000223 2-Octaprenyl-6-hydroxyphenol + 0.00026 [4Fe-4S] iron-sulfur cluster + 0.513689 L-Alanine + 0.000223 S-Adenosyl-L...
GPR,
Lower bound,0.0
Upper bound,1000.0


### Metabolites

In [9]:
# One thing to remember is the same molecule in different compartments
# is treated as a different metabolite. Also note that if you want to find the common name of the metabolite just
# look at metabolite.Name. For data.base reference got http://bigg.ucsd.edu/

E.metabolites.glc__D_c
E.metabolites.glc__D_p
E.metabolites.glc__D_e

Metabolite identifier,glc__D_e
Name,D-Glucose
Memory address,0x7fe324008a00
Formula,C6H12O6
Compartment,e
In 3 reaction(s),"EX_glc__D_e, GLCtex_copy1, GLCtex_copy2"


### Genes

In [10]:
#Each reaction has a gene reaction reactions that lists the boolean logic mapping gene  to reaction presence/absence

#For example 
E.reactions.PYK #has 2 alterantive genes
E.genes.b1854
E.genes.b1676

Gene identifier,b1676
Name,pykF
Memory address,0x7fe324375d00
Functional,True
In 1 reaction(s),PYK


### FBA basics

In [11]:
#So i'm going to run through some example FBA simulations just to illustrate the functionality 
# of the cobrapy package

#list of exchange reactions fo rthe ions
ion_reactions = ['EX_ca2_e', 'EX_cl_e','EX_cobalt2_e','EX_cu2_e',
                 'EX_fe2_e','EX_fe3_e','EX_h_e','EX_h2o_e','EX_k_e',
                 'EX_mg2_e','EX_mn2_e','EX_mobd_e','EX_na1_e','EX_nh4_e',
                 'EX_ni2_e','EX_o2_e','EX_pi_e','EX_so4_e','EX_zn2_e',
                 'EX_tungs_e','EX_sel_e','EX_slnt_e','EX_cbl1_e']
#All exhcange reactions have a lower bound of 0 except for the ions
for x in E.reactions:
    if 'EX_' in x.id:
        if x.id in ion_reactions:
            x.lower_bound = -1000.0
        else:
            x.lower_bound =0.0
            
#Note that becuase this is a pain to code up each time a shortcut for doing this is included in the CAFBAFY module
E.set_minimal_media()

In [12]:
#The model is infeasible because there is no carbon source
E.optimize()

/Users/mrebolleda/opt/miniconda3/lib/python3.9/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


<Solution infeasible at 0x7fe3294f4b20>

In [13]:
#now we add glucose (the vmax for glucose is roughly 10 so we typically use that as the glucose uptake rate 
# in fact we normally just set any carbon source as -10.0
E.reactions.EX_glc__D_e.lower_bound=-10.0
E.optimize()

,fluxes,reduced_costs
EX_cm_e,0.000000,0.000000e+00
EX_cmp_e,0.000000,-2.965572e-01
EX_co2_e,19.675223,0.000000e+00
EX_cobalt2_e,-0.000025,1.727923e-12
DM_4crsol_c,0.000219,0.000000e+00
...,...,...
RNDR4,0.000000,-2.073827e-03
RNDR4b,0.000000,-2.073827e-03
RNTR1c2,0.025705,0.000000e+00
RNTR2c2,0.026541,5.551115e-17


In [14]:
# The output of E.optimize is a pandas dataframe containing the flux through every reaction. 
# The flux for a reaction variable is the difference of the primal values for the forward and reverse reaction variables.
# you can save this to a csv file. If you just want to  quickly look at 
# uptake and secretion use
E.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.005113,0,0.00%
cl_e,EX_cl_e,0.005113,0,0.00%
cobalt2_e,EX_cobalt2_e,2.456E-05,0,0.00%
cu2_e,EX_cu2_e,0.0006965,0,0.00%
fe2_e,EX_fe2_e,0.01578,0,0.00%
glc__D_e,EX_glc__D_e,10,6,100.00%
k_e,EX_k_e,0.1918,0,0.00%
mg2_e,EX_mg2_e,0.008522,0,0.00%
mn2_e,EX_mn2_e,0.0006788,0,0.00%
mobd_e,EX_mobd_e,0.0001267,0,0.00%


In [22]:
# In_fluxes - uptake
# Out_fluxes - secretions 

# We can also inspect particular metabolites and their contributions:
E.metabolites.atp_c.summary()

In [17]:
#Note normal FBA is not guaranteed to give a unique solution (there are often multiple equally optimal solutions)
#However they often give you hugely different total fluxes. You can explore this using Flux varaibility analysis
#FVA which constrains the objective (biomass) and explores the range of flux that could give meet that constrained 
# i.e 
E.summary(fva=1) #Fluxes that give 100% of biomass
E.summary(fva=0.95) #Fluxes that give 95% of biomass

In [15]:
#You are more likely to get a unique solution using pFBA. pFBA does 2 optimizations. Firstly it optimizes growth
# and then it fixes growth and finds the minimum flux distribution that would give that growth.
cobra.flux_analysis.pfba(E)
#If all you want to know is does it grow or does it not, FBA is fine. If you want to look at fluxes/secretions
#do pFBA FBA is often easier to start with as the data remains stored in the model object so we'll stick to that for now

E.optimize()

,fluxes,reduced_costs
EX_cm_e,0.000000,0.000000
EX_cmp_e,0.000000,-0.304853
EX_co2_e,19.675223,0.000000
EX_cobalt2_e,-0.000025,0.000000
DM_4crsol_c,0.000219,0.000000
...,...,...
RNDR4,0.000000,-0.002074
RNDR4b,0.000000,-0.002074
RNTR1c2,0.025705,0.000000
RNTR2c2,0.026541,0.000000


In [16]:
#so were going to look at the simplest possible analysis using FBA. Simulating the effect of a reaction deletion.
#we are going to focus on enolase which is a key step in glycolysis that converts 2-phosphoglycerate to phosphoenolpyruvate
E.reactions.ENO

Reaction identifier,ENO
Name,Enolase
Memory address,0x7fe324b40490
Stoichiometry,2pg_c <=> h2o_c + pep_c D-Glycerate 2-phosphate <=> H2O H2O + Phosphoenolpyruvate
GPR,b2779
Lower bound,-1000.0
Upper bound,1000.0


In [25]:
#IT's often helpful not just to look at reactions but too look at metabolites. before deleting the reactions
#lets  look at what happening to phosphoenolpyruvate
E.metabolites.get_by_id('pep_c').summary()

In [17]:
#One way to knock out enolase is to simply set the lower and upper bounds to 0
E.reactions.ENO.lower_bound =-0.0
E.reactions.ENO.upper_bound =0.0
cobra.flux_analysis.pfba(E)
#If all you want to know is does it grow or does it not, FBA is fine. If you want to look at fluxes/secretions
#do pFBA FBA is often easier to start with as the data remains stored in the model object so we'll stick to that for now

E.optimize()


,fluxes,reduced_costs
EX_cm_e,0.000000,0.000000
EX_cmp_e,0.000000,-0.295953
EX_co2_e,22.853748,0.000000
EX_cobalt2_e,-0.000023,0.000000
DM_4crsol_c,0.000202,0.000000
...,...,...
RNDR4,0.000000,-0.000537
RNDR4b,0.000000,-0.000537
RNTR1c2,0.023679,0.000000
RNTR2c2,0.024449,0.000000


In [27]:
#Hmm so there is some growth cost (0.9 vs 0.98) but it's not much. Doesnt look like
# enolase is ssential. how come. To explore mecahnisticallly why the gene is not needed
# let look at2-phosphoglycerate fluxes
E.metabolites.get_by_id('pep_c').summary()

In [28]:
#so whereas previously pep_c was mostly being generated using Enolase, now it's being generated using Phosphoenolpyruvate synthase
#what if we knock this reaction out . Just to show you how this works were going to do this in a different
# way. rather than knocking out the reaction by changing the bounds we can just 
# remove the gene (in this case only 1 gene is involved)
c.manipulation.delete_model_genes(E,E.genes.b1702)

In [19]:
E.optimize()
E.summary()

In [20]:
#you can see the change in the state of gene which is now listed as non-functional
E.genes.b1702 

Gene identifier,b1702
Name,ppsA
Memory address,0x07f415f814c50
Functional,False
In 1 reaction(s),PPS


In [21]:
#still growing as well as before how come?

E.metabolites.get_by_id('pep_c').summary()

In [22]:
#Ok so PPCK is now producing pep_c. let knock that one out as well
E.reactions.PPCK.upper_bound=0.0
E.optimize()

,fluxes,reduced_costs
DM_4crsol_c,0.0,0.0
DM_5drib_c,0.0,0.0
DM_aacald_c,0.0,0.0
DM_amob_c,0.0,0.0
DM_mththf_c,0.0,0.0
...,...,...
ZN2abcpp,0.0,0.0
ZN2t3pp,0.0,0.0
ZN2tpp,0.0,0.0
ZNabcpp,0.0,0.0


In [23]:
#Ok so with those three knockouts the model can't grow. Lets finally show that restoring at least one of those 
#genes is sufficient to restore growth (a nice little example of epistasis). Now it's often the case that i might want to edit 
# a model temporarily to see an output but
#not permanently. I.e i want to add each of the genes back to the model, but only temporarily to test whetehr
#they can restore growth. you can use the 'context manner to do this very easily'

In [24]:
with E as temp_E:
    temp_E.reactions.ENO.upper_bound =1000.0
    print(temp_E.optimize()) #Enolase restores growth to triple knckout

with E as temp_E:
    c.manipulation.undelete_model_genes(temp_E) 
    # The nice thing about deleting genes is you can just undo them. However you have to use cobrapy built in functions
    # to do this (simply changing the status of the gene from function to nonfunctional will not update the reactions
    print(temp_E.slim_optimize()) # restoring the gene for PPS restores growth (slim optmize is a faster verison of optimize that only outputs the biomass function)

with E as temp_E:
    temp_E.reactions.PPCK.upper_bound =1000.0
    print(temp_E.slim_optimize()) #PPCK restores growth (slim optmize is a faster verison of optimize that only outputs the biomass function)
    

<Solution 0.982 at 0x7f415f5fe390>
0.9049381797280712
0.9047367679556663
